# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
import matplotlib.pyplot as plt

In [2]:
# введем параметр с текущей датой для сравнения
CURRENT_DATE = '06/01/2021'

In [3]:
df = pd.read_csv('main_task_new.csv')
df.head(5)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [4]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [5]:
# проверим типы данных

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
Restaurant_id        40000 non-null object
City                 40000 non-null object
Cuisine Style        30717 non-null object
Ranking              40000 non-null float64
Rating               40000 non-null float64
Price Range          26114 non-null object
Number of Reviews    37457 non-null float64
Reviews              40000 non-null object
URL_TA               40000 non-null object
ID_TA                40000 non-null object
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [6]:
# приведенем имена столбцов к виду без пробелов

df.columns = ['Restaurant_id', 'City', 'Cuisine_style', 'Ranking', 'Rating', 'Price_range', 'Number_of_reviews',
              'Reviews', 'URL_TA', 'ID_TA']

In [7]:
# количество городов, представленых в файле
df['City'].value_counts().count()

31

In [8]:
# Найдем количество уникалных значений кухонь

# функция для чистки мусора в списках кухонь
def to_list(x):
    c_list = x.replace('[', '').replace(']', '').replace("'", '').replace(', ', ',').split(',')
    return c_list

In [9]:

list_of_lists = df[~df.Cuisine_style.isna()]['Cuisine_style'].apply(to_list)
cuisine_list = []
for i in list_of_lists:
    cuisine_list.extend(i)
cuisine_list_fin = set(cuisine_list)  #уберем повторы
len(cuisine_list_fin)


125

In [10]:
# Найдем 10 самых популярных кухонь, которые есть в большинстве ресторанов
import collections
from collections import Counter
Dict = Counter(cuisine_list)
print(Dict.most_common(10))

[('Vegetarian Friendly', 11189), ('European', 10060), ('Mediterranean', 6277), ('Italian', 5964), ('Vegan Options', 4486), ('Gluten Free Options', 4113), ('Bar', 3297), ('French', 3190), ('Asian', 3011), ('Pizza', 2849)]


In [11]:
# сгенерируем новый признак исходя из количества представленных кухонь в каждом ресторане:

def count_cuis(c_list):
    count = 0
    for row in c_list:
        if row != 'none':
            count += 1
        else:
            count = 1
    return count


df['Cuisine_style'] = df['Cuisine_style'].fillna('Vegetarian Friendly')#заполним пропуски наиболее популярной кухней  
df['Cuisine_style'] = df['Cuisine_style'].apply(to_list)
df['cuisine_count'] = df['Cuisine_style'].apply(count_cuis)


In [12]:
#посчитаем среднее количество кухонь в ресторане как отношение суммы всех кухонь к количеству ресторанов

mean_coisine_number = df.cuisine_count.sum()/df.Restaurant_id.count()
mean_coisine_number

2.6224

# Дата

In [13]:

# очищаем данные
df['Review Date'] = df['Reviews'].str[2:-2].str.split("\], \[").str[1].replace('\'', '', regex=True)
# Добавляем признак, есть ли отзыв
df['Is Reviewed'] = df['Review Date'].apply(lambda x: 0 if x == '' else 1)

In [14]:
# Заполняем пропущенные значения и делаем списки из дат
df['Review Date'] = df['Review Date'].replace('', np.nan)
df['Review Date'] = df['Review Date'].str.split(", ")

In [15]:
# Создаем два новых атрибута - количество дней между отзывами 
# и количество дней с момента последнего отзыва
def datediff (dateslist):
    return abs(max( [dt.strptime(d, '%m/%d/%Y') for d in dateslist]) 
               - min( [dt.strptime(d, '%m/%d/%Y') for d in dateslist])).days

def dayspassed (dateslist):
    return abs(max( [dt.strptime(d, '%m/%d/%Y') for d in dateslist])
              - dt.strptime(CURRENT_DATE, '%m/%d/%Y')).days
    
df['Days Between'] = df['Review Date'][~df['Review Date'].isna()].apply(lambda d: datediff(d))
df['Days Since'] = df['Review Date'][~df['Review Date'].isna()].apply(lambda d: dayspassed(d))


In [16]:
# заполняем пустые значния 0, так как можно считать, 
# что если отзыва нет, то человек просто поставил оценку ресторану
df['Days Between'] = df['Days Between'].fillna(0)
df['Days Since'] = df['Days Since'].fillna(0)

In [17]:
df.head(5)

,Restaurant_id,City,Cuisine_style,Ranking,Rating,Price_range,Number_of_reviews,Reviews,URL_TA,ID_TA,cuisine_count,Review Date,Is Reviewed,Days Between,Days Since
0,id_5569,Paris,"[European, French, International]",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,3,"[12/31/2017, 11/20/2017]",1,41.0,1248.0
1,id_1535,Stockholm,[Vegetarian Friendly],1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,1,"[07/06/2017, 06/19/2016]",1,382.0,1426.0
2,id_352,London,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,7,"[01/08/2018, 01/06/2018]",1,2.0,1240.0
3,id_3456,Berlin,[Vegetarian Friendly],3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,1,NaN,0,0.0,0.0
4,id_615,Munich,"[German, Central European, Vegetarian Friendly]",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,3,"[11/18/2017, 02/19/2017]",1,272.0,1291.0


In [18]:
# находим максимальное количество дней между отзывами
df['Days Between'].max()

3207.0

In [19]:
# Заменим количественные показатели на числовые для диапазона цен и заполним пропуски на значение "2", как самое встречающееся
df['Price_range'] = df['Price_range'].map({'$':1,'$$ - $$$':2,'$$$$':3}).fillna(2)

In [20]:
# Cоздадим новый параметр - количество ресторанов в одном городе
rest_count = df.groupby('City')['Restaurant_id'].count().to_dict()
df['Restaurants in City'] = df['City'].map(rest_count)

# Добавим новые признаки с внешними данными -  население городов
populations = {
    'London': 8567000, 
    'Paris':  9904000, 
    'Madrid':  5567000,
    'Barcelona':  4920000,
    'Berlin':  3406000,
    'Milan':  2945000,
    'Rome':  3339000,
    'Prague':  1162000,
    'Lisbon':  2812000,
    'Vienna':  2400000,
    'Amsterdam':  1031000,
    'Brussels':  1743000,
    'Hamburg':  1757000,
    'Munich':  1275000,
    'Lyon':  1423000,
    'Stockholm':  1264000,
    'Budapest':  1679000,
    'Warsaw':  1707000,
    'Dublin':  1059000,
    'Copenhagen':  1085000,
    'Athens':  3242000,
    'Edinburgh':  504966,
    'Zurich':  1108000,
    'Oporto':  1337000,
    'Geneva':  1240000,
    'Krakow':  756000,
    'Oslo':  835000,
    'Helsinki':  1115000,
    'Bratislava':  423737,
    'Luxembourg':  107260,
    'Ljubljana':  314807,
}


df['Population'] = df['City'].map(populations)

df.head(5)

,Restaurant_id,City,Cuisine_style,Ranking,Rating,Price_range,Number_of_reviews,Reviews,URL_TA,ID_TA,cuisine_count,Review Date,Is Reviewed,Days Between,Days Since,Restaurants in City,Population
0,id_5569,Paris,"[European, French, International]",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,3,"[12/31/2017, 11/20/2017]",1,41.0,1248.0,4897,9904000
1,id_1535,Stockholm,[Vegetarian Friendly],1537.0,4.0,2.0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,1,"[07/06/2017, 06/19/2016]",1,382.0,1426.0,820,1264000
2,id_352,London,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,7,"[01/08/2018, 01/06/2018]",1,2.0,1240.0,5757,8567000
3,id_3456,Berlin,[Vegetarian Friendly],3458.0,5.0,2.0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,1,NaN,0,0.0,0.0,2155,3406000
4,id_615,Munich,"[German, Central European, Vegetarian Friendly]",621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,3,"[11/18/2017, 02/19/2017]",1,272.0,1291.0,893,1275000


In [21]:
# Заполним пропуски в количество оценок  медианным значением

df['Number_of_reviews'] = df['Number_of_reviews'].fillna(
    df['Number_of_reviews'].mean())

In [22]:
# убираем лишние столбцы
df= df.drop(['City', 'Cuisine_style', 'Reviews', 'URL_TA', 'ID_TA', 'Review Date', 'Is Reviewed', 'Days Since'], axis=1)
df

,Restaurant_id,Ranking,Rating,Price_range,Number_of_reviews,cuisine_count,Days Between,Restaurants in City,Population
0,id_5569,5570.0,3.5,2.0,194.0,3,41.0,4897,9904000
1,id_1535,1537.0,4.0,2.0,10.0,1,382.0,820,1264000
2,id_352,353.0,4.5,3.0,688.0,7,2.0,5757,8567000
3,id_3456,3458.0,5.0,2.0,3.0,1,0.0,2155,3406000
4,id_615,621.0,4.0,2.0,84.0,3,272.0,893,1275000
...,...,...,...,...,...,...,...,...,...
39995,id_499,500.0,4.5,2.0,79.0,4,34.0,2133,2945000
39996,id_6340,6341.0,3.5,2.0,542.0,5,9.0,4897,9904000
39997,id_1649,1652.0,4.5,2.0,4.0,2,3127.0,820,1264000
39998,id_640,641.0,4.0,2.0,70.0,5,23.0,727,1707000


In [23]:
# проверяем что осталось в датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 9 columns):
Restaurant_id          40000 non-null object
Ranking                40000 non-null float64
Rating                 40000 non-null float64
Price_range            40000 non-null float64
Number_of_reviews      40000 non-null float64
cuisine_count          40000 non-null int64
Days Between           40000 non-null float64
Restaurants in City    40000 non-null int64
Population             40000 non-null int64
dtypes: float64(5), int64(3), object(1)
memory usage: 2.7+ MB


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [24]:
df.head(5)

,Restaurant_id,Ranking,Rating,Price_range,Number_of_reviews,cuisine_count,Days Between,Restaurants in City,Population
0,id_5569,5570.0,3.5,2.0,194.0,3,41.0,4897,9904000
1,id_1535,1537.0,4.0,2.0,10.0,1,382.0,820,1264000
2,id_352,353.0,4.5,3.0,688.0,7,2.0,5757,8567000
3,id_3456,3458.0,5.0,2.0,3.0,1,0.0,2155,3406000
4,id_615,621.0,4.0,2.0,84.0,3,272.0,893,1275000


In [25]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis=1)
y = df['Rating']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 9 columns):
Restaurant_id          40000 non-null object
Ranking                40000 non-null float64
Rating                 40000 non-null float64
Price_range            40000 non-null float64
Number_of_reviews      40000 non-null float64
cuisine_count          40000 non-null int64
Days Between           40000 non-null float64
Restaurants in City    40000 non-null int64
Population             40000 non-null int64
dtypes: float64(5), int64(3), object(1)
memory usage: 2.7+ MB


In [26]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [27]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [28]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [29]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [30]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21328900000000003
